In [1]:
## 모델다운로드 - 실행되면 intel 폴더가 생기고 그 안에 모델이 다운로드되어있다.
## 한번만 실행해준다.
! omz_downloader --name face-detection-adas-0001 --precision FP16

################|| Downloading face-detection-adas-0001 ||################

========== Downloading C:\BrainAI\Day5_face_detection\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.xml
... 100%, 304 KB, 309 KB/s, 0 seconds passed

========== Downloading C:\BrainAI\Day5_face_detection\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.bin
... 49%, 1024 KB, 618 KB/s, 1 seconds passed
... 99%, 2048 KB, 757 KB/s, 2 seconds passed
... 100%, 2056 KB, 756 KB/s, 2 seconds passed



In [2]:
import openvino as ov
import cv2
import numpy as np

import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
core = ov.Core()
options=core.available_devices

options

['CPU', 'GPU', 'NPU']

## 로드

In [4]:
core=ov.Core()

model=core.read_model(model="models/face-detection-adas-0001.xml")
face_model=core.compile_model(model=model, device_name="CPU")

face_input_layer=face_model.input(0)
face_output_layer=face_model.output(0)

print("Input layer shape:", face_input_layer.shape)
print("Onput layer shape:", face_output_layer.shape)

Input layer shape: [1,3,384,672]
Onput layer shape: [1,1,200,7]


## 추론

In [ ]:
## 오픈비노 도큐먼트 확인
## https://docs.openvino.ai/2024/omz_models_model_face_detection_adas_0001.html

In [5]:
frame = cv2.imread("images/test.jpg")
resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
transposed_frame = resized_frame.transpose(2, 0, 1)
input_frame = np.expand_dims(transposed_frame, 0)

In [6]:
face_output = face_model([input_frame])[face_output_layer]

## 출력

In [7]:
def DrawBoundingBoxes(output, image, conf):

    canvas = image.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기
    
   
    return canvas

In [10]:
canvas = DrawBoundingBoxes(face_output, frame, conf=0.5)

cv2.imshow("Canvas", canvas)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 백그라운드 이미지 또는 영상 출력을 위한 함수

In [16]:
def AddBackground(frame, bg):

    frame_h, frame_w = frame.shape[0], frame.shape[1]
    new_h = 500
    new_w = int((new_h/frame_h)*frame_w)
    frame_resize = cv2.resize(frame, (new_w, new_h))

    xmax = bg.shape[1] - 350  # x좌표위치를 설정함.
    ymax = bg.shape[0] - 175  # y좌표위치를 설정함.
    xmin = xmax - new_w
    ymin = ymax - new_h

    bg[ymin:ymax, xmin:xmax] = frame_resize

    return bg

## 백그라운드 + 이미지 출력

In [14]:
background = "./images/Background.jpg"  #사용할 배경화면 경로
canvas = DrawBoundingBoxes(face_output, frame, conf=0.5)  
bg = cv2.imread(background)

deployment = AddBackground(canvas, bg)
cv2.imshow("Deployment", deployment)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 백그라운드 + 영상 출력

In [21]:
camera=cv2.VideoCapture(0)
bg = cv2.imread(background)
background = "./images/Background.jpg"  #사용할 배경화면 경로

while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    if ret == False:
        break
    # AI 추론을 위한 데이터 전처리
    resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0) 
    # AI 추론
    face_output = face_model([input_frame])[face_output_layer]
    # AI 추론 결과 시각화: 박스 그기기
    canvas = DrawBoundingBoxes(face_output, frame, conf=0.5)
    deployment = AddBackground(canvas, bg)
    cv2.imshow('Press Spacebar to Exit', deployment)
    # 영상 종료: 스페이스 바
    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break
        
camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()